In [1]:
import numpy as np
import os
import pandas as pd
import sys
import torch

from matplotlib import pyplot as plt
from tqdm import tqdm

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
x = torch.arange(9).reshape(3,3)
idx = torch.tril_indices(3, 3, offset=-1)

# Usa los índices para obtener los elementos de la parte inferior triangular
lower_triangular_elements = x[idx[0], idx[1]]
x[idx[0], idx[1]] = -2
print(lower_triangular_elements)
print(x)

tensor([3, 6, 7])
tensor([[ 0,  1,  2],
        [-2,  4,  5],
        [-2, -2,  8]])


In [3]:
argwhere_format = torch.stack(idx, dim=1)
argwhere_format

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [ ]:
import torch

# Genera los índices de la parte inferior triangular de una matriz de 3x3
indices = torch.tril_indices(row=3, col=3, offset=0)

# Transponer y apilar los índices para obtener el formato similar a argwhere
argwhere_format = torch.stack((indices[0], indices[1]), dim=1)
print(argwhere_format)

In [ ]:
x, y = zip(indices)
print(x, y)

In [4]:
import torch

from VAE.nn.VariationalLayer import VariationalLayer
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch import nn
import numpy as np

class AnomalyDetector(nn.Linear, VariationalLayer):
    '''
        This implementation is for anomaly detection.
    '''
    def __init__(self, in_features, out_features, bias=True, full_cov=False, sigma_anomaly=3) -> None:
        assert out_features > 1, "The Gaussian posterior variables must be greater than 1"

        super(AnomalyDetector, self).__init__(in_features, out_features, bias)
        self.log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
        self.log_sigma_bias = Parameter(torch.Tensor(out_features))

        if full_cov:
            # Correlation estimation
            self.corr_weight = Parameter(torch.Tensor(out_features, in_features))
            self.corr_bias = Parameter(torch.Tensor(out_features))
        
        torch.nn.init.xavier_uniform_(self.log_sigma_weight)
        self.log_sigma_bias.data.fill_(2*np.log((2*sigma_anomaly)))
        
        self.sigma_anomaly = sigma_anomaly
        self.mu, self.sigma = None, None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mu = F.linear(x, self.weight, self.bias) 
        sigma = torch.exp(0.5 * F.linear(x, self.log_sigma_weight, self.log_sigma_bias))

        # if self.training:
        self.mu, self.sigma = mu, sigma

        # Reparameterization trick
        eps = torch.normal(0, torch.ones_like(sigma))
        return mu + sigma * eps

    def kl_reg(self, targets: torch.Tensor) -> torch.Tensor:
        # KL-Divergence regularization
        assert torch.all((targets == 0) | (targets == 1))
        sigma_2 = (torch.ones_like(targets) + ((self.sigma_anomaly-1) * targets)).unsqueeze(1).cuda()
        result = (torch.log(sigma_2) - torch.log(self.sigma) + (self.sigma**2 + self.mu**2)/(2*sigma_2**2) - 0.5)
        
        return result.mean() 

## Correlation and Covariance Implementation

In [110]:
import math
in_features, out_features = 10, 3
corr_size = math.comb(out_features, 2)
corr_weight = Parameter(torch.Tensor(corr_size, in_features))
corr_bias = Parameter(torch.Tensor(corr_size))

log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
log_sigma_bias = Parameter(torch.Tensor(out_features))

torch.nn.init.xavier_uniform_(log_sigma_weight)
log_sigma_bias.data.fill_(2*np.log((2*1)))

torch.nn.init.xavier_uniform_(corr_weight)
corr_bias.data.fill_(0)

x = torch.rand((5, 10))

In [111]:
corr = F.tanh(F.linear(x, corr_weight, corr_bias))
sigma = torch.exp(0.5 * F.linear(x, log_sigma_weight, log_sigma_bias))

print(corr)
print(sigma)

tensor([[ 0.6322, -0.2046, -0.4174],
        [ 0.4306,  0.0737,  0.2556],
        [ 0.6470,  0.1931,  0.1220],
        [ 0.4489, -0.4777,  0.0782],
        [ 0.6865,  0.2448,  0.4651]], grad_fn=<TanhBackward0>)
tensor([[1.7624, 2.4587, 1.5704],
        [1.8130, 1.6283, 2.6903],
        [1.8902, 2.9939, 2.0489],
        [1.7350, 1.6193, 2.1700],
        [1.3757, 2.3768, 2.0024]], grad_fn=<ExpBackward0>)


In [119]:
corr.shape

torch.Size([5, 3])

In [112]:
cov1 = corr[:, 0] * (sigma[:, 0] * sigma[:, 1])
cov2 = corr[:, 1] * (sigma[:, 0] * sigma[:, 2])
cov3 = corr[:, 2] * (sigma[:, 1] * sigma[:, 2])
cov1, cov2, cov3

(tensor([2.7396, 1.2711, 3.6612, 1.2610, 2.2448], grad_fn=<MulBackward0>),
 tensor([-0.5663,  0.3595,  0.7477, -1.7986,  0.6743], grad_fn=<MulBackward0>),
 tensor([-1.6115,  1.1198,  0.7485,  0.2746,  2.2134], grad_fn=<MulBackward0>))

In [113]:
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)

a = torch.zeros((1,3,3))
a[:, *tuple(idx)] = 1
a

tensor([[[0., 0., 0.],
         [1., 0., 0.],
         [1., 1., 0.]]])

In [114]:
print(*tuple(idx))

tensor([1, 2, 2]) tensor([0, 0, 1])


In [115]:
idx

tensor([[1, 2, 2],
        [0, 0, 1]])

In [124]:
# idx = list(combinations(range(out_features), 2))
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
tuple(idx)

(tensor([1, 2, 2]), tensor([0, 0, 1]))

In [125]:
corr.size()

torch.Size([5, 3])

In [130]:
corr_matrix = torch.zeros((corr.size(0), out_features, out_features))
corr_matrix[:, *tuple(idx)] = corr
corr_matrix

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.6322,  0.0000,  0.0000],
         [-0.2046, -0.4174,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.4306,  0.0000,  0.0000],
         [ 0.0737,  0.2556,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.6470,  0.0000,  0.0000],
         [ 0.1931,  0.1220,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.4489,  0.0000,  0.0000],
         [-0.4777,  0.0782,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.6865,  0.0000,  0.0000],
         [ 0.2448,  0.4651,  0.0000]]], grad_fn=<CopySlices>)

In [181]:
def covariance_matrix(corr, sigma):
    idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
    cov_matrix = torch.zeros((corr.size(0), corr.size(1), corr.size(1)))

    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    return cov_matrix

In [182]:
covariance_matrix(corr, sigma)

tensor([[[ 3.1061,  0.0000,  0.0000],
         [ 2.7396,  6.0451,  0.0000],
         [-0.5663, -1.6115,  2.4663]],

        [[ 3.2869,  0.0000,  0.0000],
         [ 1.2711,  2.6515,  0.0000],
         [ 0.3595,  1.1198,  7.2378]],

        [[ 3.5728,  0.0000,  0.0000],
         [ 3.6612,  8.9634,  0.0000],
         [ 0.7477,  0.7485,  4.1978]],

        [[ 3.0103,  0.0000,  0.0000],
         [ 1.2610,  2.6220,  0.0000],
         [-1.7986,  0.2746,  4.7089]],

        [[ 1.8924,  0.0000,  0.0000],
         [ 2.2448,  5.6493,  0.0000],
         [ 0.6743,  2.2134,  4.0096]]], grad_fn=<AddBackward0>)

In [131]:
print(corr)

tensor([[ 0.6322, -0.2046, -0.4174],
        [ 0.4306,  0.0737,  0.2556],
        [ 0.6470,  0.1931,  0.1220],
        [ 0.4489, -0.4777,  0.0782],
        [ 0.6865,  0.2448,  0.4651]], grad_fn=<TanhBackward0>)


In [174]:
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
test = sigma[:, idx.T]

cov = test.prod(dim=-1) * corr

corr.shape, test.shape, cov.shape

(torch.Size([5, 3]), torch.Size([5, 3, 2]), torch.Size([5, 3]))

In [175]:
test, test.prod(dim=-1)

(tensor([[[2.4587, 1.7624],
          [1.5704, 1.7624],
          [1.5704, 2.4587]],
 
         [[1.6283, 1.8130],
          [2.6903, 1.8130],
          [2.6903, 1.6283]],
 
         [[2.9939, 1.8902],
          [2.0489, 1.8902],
          [2.0489, 2.9939]],
 
         [[1.6193, 1.7350],
          [2.1700, 1.7350],
          [2.1700, 1.6193]],
 
         [[2.3768, 1.3757],
          [2.0024, 1.3757],
          [2.0024, 2.3768]]], grad_fn=<IndexBackward0>),
 tensor([[4.3332, 2.7678, 3.8612],
         [2.9521, 4.8775, 4.3808],
         [5.6590, 3.8727, 6.1341],
         [2.8095, 3.7650, 3.5138],
         [3.2697, 2.7546, 4.7594]], grad_fn=<ProdBackward1>))

In [177]:
2.4587 * 1.7624, 1.5704 * 1.7624, 1.5704 * 2.4587

(4.33321288, 2.76767296, 3.86114248)

In [166]:
# idx = torch.triu_indices(3, 3, offset=1)
idx = torch.tril_indices(3, 3, offset=-1)
test = sigma[:, idx.T]
test

tensor([[[2.4587, 1.7624],
         [1.5704, 1.7624],
         [1.5704, 2.4587]],

        [[1.6283, 1.8130],
         [2.6903, 1.8130],
         [2.6903, 1.6283]],

        [[2.9939, 1.8902],
         [2.0489, 1.8902],
         [2.0489, 2.9939]],

        [[1.6193, 1.7350],
         [2.1700, 1.7350],
         [2.1700, 1.6193]],

        [[2.3768, 1.3757],
         [2.0024, 1.3757],
         [2.0024, 2.3768]]], grad_fn=<IndexBackward0>)

In [150]:
sigma[0, [[0,1], [0,2], [1,2]]]

tensor([[1.7624, 2.4587],
        [1.7624, 1.5704],
        [2.4587, 1.5704]], grad_fn=<IndexBackward0>)

In [266]:
def correlation_tests():
    # sigma = torch.ones(1,5)*2
    sigma = torch.arange(1, 6).reshape(1, 5)
    corr = torch.tensor([[1, .5, .25, .2, .1, .09, .08, .05, .02, .01]])

    idx = torch.tril_indices(5, 5, offset=-1)

    print(tuple(idx))
    corr_matrix = torch.zeros((1, 5, 5))
    corr_matrix[:, *tuple(idx)] = corr
    print(corr_matrix.shape, torch.diag_embed(sigma).shape)
    print((corr_matrix + torch.eye(*corr_matrix.shape[1:])))


    cov_matrix = torch.zeros((1, 5, 5))
    # cov_10 = corr_matrix[:, 1, 0] * (sigma[0, 1] * sigma[0, 0])
    # cov_20 = corr_matrix[:, 2, 0] * (sigma[0, 2] * sigma[0, 0])
    # cov_21 = corr_matrix[:, 2, 1] * (sigma[0, 2] * sigma[0, 1])
    # cov_30 = corr_matrix[:, 3, 0] * (sigma[0, 3] * sigma[0, 0])
    # cov_31 = corr_matrix[:, 3, 1] * (sigma[0, 3] * sigma[0, 1])
    # cov_32 = corr_matrix[:, 3, 2] * (sigma[0, 3] * sigma[0, 2])

    for i in range(1, 5):
        for j in range(i):
            cov_matrix[:, i, j] = corr_matrix[:, i, j] * (sigma[0, i] * sigma[0, j])

    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    print(cov_matrix)
    
    return sigma, corr, cov_matrix

def correlation_tests(n_samples=1, n_variables=3):
    '''
        Inneficent way to generate the correlation matrix. This function
        is only for testing purposes.
    '''
    sigma = torch.arange(1, (n_samples*n_variables)+1).reshape(n_samples, n_variables)
    
    n_corr = math.comb(n_variables, 2)
    corr = torch.linspace(-1, 1, steps=n_samples*n_corr).reshape(n_samples, n_corr)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    corr_matrix = torch.zeros((n_samples, n_variables, n_variables))
    for n in range(n_samples):
        corr_matrix[n, *tuple(idx)] = corr[n]

    cov_matrix = torch.zeros((n_samples, n_variables, n_variables))

    for n in range(n_samples):
        for i in range(1, n_variables):
            for j in range(i):
                corr_matrix[n, i, j]
                cov_matrix[n, i, j] = corr_matrix[n, i, j] * (sigma[n, i] * sigma[n, j])

    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
        
    return sigma, corr, cov_matrix

sigma, corr, cov_matrix = correlation_tests(n_samples=3, n_variables=5)


cov_matrix

tensor([[[  1.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ -2.0000,   4.0000,   0.0000,   0.0000,   0.0000],
         [ -2.7931,  -5.1724,   9.0000,   0.0000,   0.0000],
         [ -3.1724,  -5.7931,  -7.8621,  16.0000,   0.0000],
         [ -2.9310,  -5.1724,  -6.7241,  -7.5862,  25.0000]],

        [[ 36.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [-13.0345,  49.0000,   0.0000,   0.0000,   0.0000],
         [-11.5862,  -9.6552,  64.0000,   0.0000,   0.0000],
         [ -5.5862,  -2.1724,   2.4828,  81.0000,   0.0000],
         [  6.2069,  12.0690,  19.3103,  27.9310, 100.0000]],

        [[121.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ 50.0690, 144.0000,   0.0000,   0.0000,   0.0000],
         [ 64.1034,  80.6897, 169.0000,   0.0000,   0.0000],
         [ 90.2759, 110.0690, 131.7931, 196.0000,   0.0000],
         [130.8621, 155.1724, 181.5517, 210.0000, 225.0000]]])

In [271]:
1 * math.sqrt(196*225)

210.0

In [268]:
corr, sigma

(tensor([[-1.0000, -0.9310, -0.8621, -0.7931, -0.7241, -0.6552, -0.5862, -0.5172,
          -0.4483, -0.3793],
         [-0.3103, -0.2414, -0.1724, -0.1034, -0.0345,  0.0345,  0.1034,  0.1724,
           0.2414,  0.3103],
         [ 0.3793,  0.4483,  0.5172,  0.5862,  0.6552,  0.7241,  0.7931,  0.8621,
           0.9310,  1.0000]]),
 tensor([[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10],
         [11, 12, 13, 14, 15]]))

In [269]:
def covariance_matrix(corr, sigma):
    n_variables = sigma.size(1)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    cov_matrix = torch.zeros((corr.size(0), n_variables, n_variables))
    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    return cov_matrix

In [270]:
cov_matrix_opt = covariance_matrix(corr, sigma)

torch.all(cov_matrix == cov_matrix_opt)

tensor(True)

In [249]:
0.6 * math.sqrt(16*36)

14.399999999999999

### ---

In [ ]:
import math
in_features, out_features = 10, 3
corr_size = math.comb(out_features, 2)
corr_weight = Parameter(torch.Tensor(corr_size, in_features))
corr_bias = Parameter(torch.Tensor(corr_size))

log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
log_sigma_bias = Parameter(torch.Tensor(out_features))

torch.nn.init.xavier_uniform_(log_sigma_weight)
log_sigma_bias.data.fill_(2*np.log((2*1)))

torch.nn.init.xavier_uniform_(corr_weight)
corr_bias.data.fill_(0)

x = torch.rand((5, 10))

In [274]:
corr = F.tanh(F.linear(x, corr_weight, corr_bias))
sigma = torch.exp(0.5 * F.linear(x, log_sigma_weight, log_sigma_bias))

print(corr)
print(sigma**2)

tensor([[ 0.6322, -0.2046, -0.4174],
        [ 0.4306,  0.0737,  0.2556],
        [ 0.6470,  0.1931,  0.1220],
        [ 0.4489, -0.4777,  0.0782],
        [ 0.6865,  0.2448,  0.4651]], grad_fn=<TanhBackward0>)
tensor([[3.1061, 6.0451, 2.4663],
        [3.2869, 2.6515, 7.2378],
        [3.5728, 8.9634, 4.1978],
        [3.0103, 2.6220, 4.7089],
        [1.8924, 5.6493, 4.0096]], grad_fn=<PowBackward0>)


In [273]:
def covariance_matrix(corr, sigma):
    n_variables = sigma.size(1)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    cov_matrix = torch.zeros((corr.size(0), n_variables, n_variables))
    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    return cov_matrix

cov_matrix = covariance_matrix(corr, sigma)
cov_matrix

tensor([[[ 3.1061,  0.0000,  0.0000],
         [ 2.7396,  6.0451,  0.0000],
         [-0.5663, -1.6115,  2.4663]],

        [[ 3.2869,  0.0000,  0.0000],
         [ 1.2711,  2.6515,  0.0000],
         [ 0.3595,  1.1198,  7.2378]],

        [[ 3.5728,  0.0000,  0.0000],
         [ 3.6612,  8.9634,  0.0000],
         [ 0.7477,  0.7485,  4.1978]],

        [[ 3.0103,  0.0000,  0.0000],
         [ 1.2610,  2.6220,  0.0000],
         [-1.7986,  0.2746,  4.7089]],

        [[ 1.8924,  0.0000,  0.0000],
         [ 2.2448,  5.6493,  0.0000],
         [ 0.6743,  2.2134,  4.0096]]], grad_fn=<AddBackward0>)